# Combination of CSVs
- Due to the sheer size of the dataset (a total of 9 tweet data sets), we had to run the Feature Engineering script for each of the 9 tweet data sets and obtain the relevant information required from each dataset. 
- After which, we are able to concatenate them into two csv files `graph.csv` and `user.csv`.

## 1.1 Importing Dependencies

In [ ]:
import pandas as pd
from pymongo import MongoClient

## 1.2 Accessing all the relevant user information from the respective CSVs

In [17]:
user_0 = pd.read_csv('user_0.csv')
user_1 = pd.read_csv('user_1.csv')
user_2 = pd.read_csv('user_2.csv')
user_3 = pd.read_csv('user_3.csv')
user_4 = pd.read_csv('user_4.csv')
user_5 = pd.read_csv('user_5.csv')
user_6 = pd.read_csv('user_6.csv')
user_7 = pd.read_csv('user_7.csv')
user_8 = pd.read_csv('user_8.csv')

In [18]:
#Stacking the CSVs together

users = pd.concat([user_0,
                user_1, 
                user_2,
                user_3,
                user_4, 
                user_5,
                user_6,
                user_7, 
                user_8])

## 1.3 Aggregating all duplicate users within the sampling procedure

In [5]:
#Aggregate all duplicate users within each sampling procedure
log = {col: 'mean' if users[col].dtype in ['float64', 'int64'] and users[col] != 'source_user_id' else 'first' for col in users.columns if col != 'source_user_id'}

users_result = users.groupby('source_user_id').agg(log).reset_index()


In [6]:
users_result["label"].value_counts()

label
0.0    59769
1.0    49656
Name: count, dtype: int64

## 2.1 Accessing all the relevant graph information from the respective CSVs

In [8]:
#Concatenate all tweet relationships

graph_0 = pd.read_csv('graph_0.csv')
graph_1 = pd.read_csv('graph_1.csv')
graph_2 = pd.read_csv('graph_2.csv')
graph_3 = pd.read_csv('graph_3.csv')
graph_4 = pd.read_csv('graph_4.csv')
graph_5 = pd.read_csv('graph_5.csv')
graph_6 = pd.read_csv('graph_6.csv')
graph_7 = pd.read_csv('graph_7.csv')
graph_8 = pd.read_csv('graph_8.csv')

In [11]:
#Stacking the tweet CSVs together

graph = pd.concat([graph_0,
                graph_1, 
                graph_2,
                graph_3,
                graph_4, 
                graph_5,
                graph_6,
                graph_7, 
                graph_8])

## 3. Save user data and graph data in CSV

In [12]:
users_result.to_csv("users.csv", index = False)
graph.to_csv("graph.csv", index = False)